# **Top 5 Queries**
## **Query 1 (Chapter 7 Exercise 1)**

### Proposition: Write a query that computes for each customer order, both a rank and a dense rank
### Table: dbo.Orders
### Columns: CustomerId, OrderId, QTY, RANK() OVER (PARTITION BY CustomerId ORDER BY QTY) AS rnk, DENSE_RANK() OVER (PARTITION BY CustomerId ORDER BY QTY) AS drnk
### Predicate: This query provides the rank and dense rank of each order's quantity within the orders of each customer by RANK() and DENSE_RANK()

### *What I think is special is that for this query, it shows us the difference between RANK() and DENSE_RANK().*



In [ ]:
USE TSQLV4
SELECT custid, orderid, qty, 
    RANK() OVER (PARTITION BY custid ORDER BY qty) AS rnk,
    DENSE_RANK() OVER (PARTITION BY custid ORDER BY qty) AS drnk
FROM dbo.Orders;

USE Northwinds2022TSQLV7;
SELECT CustomerId, OrderId, QTY, 
    RANK() OVER (PARTITION BY CustomerId ORDER BY QTY) AS rnk,
    DENSE_RANK() OVER (PARTITION BY CustomerId ORDER BY QTY) AS drnk
FROM dbo.Orders;




## **Query 2 (Chapter 7 Exercise 3)**

### Proposition: Write a query that computes difference for each customer order with previous and next order
### Table: dbo.Orders
### Columns: CustomerId, OrderId, QTY, QTY - LAG(QTY) OVER (PARTITION BY CustomerId ORDER BY OrderDate) AS diffprev, QTY - LEAD(QTY) OVER (PARTITION BY CustomerId ORDER BY OrderDate) AS diffnext
### Predicate: This query uses LAG() and LEAD() to find the previous and next order of current order, then substract it from the current order to find the difference

### *What I think is special is that for this query, it shows us the function of LAG() and LEAD().*

In [ ]:
USE TSQLV4
SELECT custid, orderid, qty,
    qty - LAG(qty) OVER (PARTITION BY custid ORDER BY orderdate) AS diffprev,
    qty - LEAD(qty) OVER (PARTITION BY custid ORDER BY orderdate) AS diffnext
FROM dbo.Orders

USE Northwinds2022TSQLV7
SELECT CustomerId, OrderId, QTY,
    QTY - LAG(QTY) OVER (PARTITION BY CustomerId ORDER BY OrderDate) AS diffprev,
    QTY - LEAD(QTY) OVER (PARTITION BY CustomerId ORDER BY OrderDate) AS diffnext
FROM dbo.Orders



## **Query 3**

### Proposition: Show the employee with their order month, value, and runval
### Table: Sales.EmpOrders
### Columns: EmployeeId, ordermonth, val, SUM(val)... AS runval
### Predicate: This query computes the cumulative sum of sales for each employee over different months

### *What I think is special is that for this query, it shows us how to generate runval.*

In [ ]:
USE TSQLV4;

SELECT empid, ordermonth, val,
  SUM(val) OVER(PARTITION BY empid
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS runval
FROM Sales.EmpOrders;

USE Northwinds2022TSQLV7;

SELECT EmployeeId, ordermonth, val,
  SUM(val) OVER(PARTITION BY EmployeeId
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS runval
FROM Sales.EmpOrders;


## **Query 4**

### Proposition: Creating a table named EmpCustOrders and populating it with data using a PIVOT operation, and finally selecting all rows from the created table
### Table: dbo.Orders
### Columns: EmployeeId, A, B, C, D
### Predicate: It transforms the data from a normalized form into a pivoted form where each employee has columns representing the quantities ordered by different customers 

### *What I think is special is that for this query, it shows us how to transform the data from a normalized form into a pivoted form.*

In [ ]:
USE TSQLV4;
DROP TABLE IF EXISTS dbo.EmpCustOrders;
CREATE TABLE dbo.EmpCustOrders
(
  empid INT NOT NULL
    CONSTRAINT PK_EmpCustOrders PRIMARY KEY,
  A VARCHAR(5) NULL,
  B VARCHAR(5) NULL,
  C VARCHAR(5) NULL,
  D VARCHAR(5) NULL
);

INSERT INTO dbo.EmpCustOrders(empid, A, B, C, D)
  SELECT empid, A, B, C, D
  FROM (SELECT empid, custid, qty
        FROM dbo.Orders) AS D
    PIVOT(SUM(qty) FOR custid IN(A, B, C, D)) AS P;

SELECT * FROM dbo.EmpCustOrders;


USE Northwinds2022TSQLV7;
DROP TABLE IF EXISTS dbo.EmpCustOrders;
CREATE TABLE dbo.EmpCustOrders
(
  empid INT NOT NULL
    CONSTRAINT PK_EmpCustOrders PRIMARY KEY,
  A VARCHAR(5) NULL,
  B VARCHAR(5) NULL,
  C VARCHAR(5) NULL,
  D VARCHAR(5) NULL
);

INSERT INTO dbo.EmpCustOrders(empid, A, B, C, D)
  SELECT EmployeeId, A, B, C, D
  FROM (SELECT EmployeeId, CustomerId, QTY
        FROM dbo.Orders) AS D
    PIVOT(SUM(QTY) FOR CustomerId IN(A, B, C, D)) AS P;

SELECT * FROM dbo.EmpCustOrders;



## **Query 5**

### Proposition: Display the employee, customer, and quantity
### Table: dbo.EmpCustOrders
### Columns: empid, custid, qty
### Predicate: It reverses the previous PIVOT operation and presents the data in its normalized form

### *What I think is special is that for this query, it shows us how to reverse the data from a pivoted form back into a normalized form.*

In [ ]:
USE TSQLV4
SELECT empid, custid, qty
FROM dbo.EmpCustOrders
  UNPIVOT(qty FOR custid IN(A, B, C, D)) AS U;

USE Northwinds2022TSQLV7
SELECT empid, custid, qty
FROM dbo.EmpCustOrders
  UNPIVOT(qty FOR custid IN(A, B, C, D)) AS U;
